In [1]:
from docx import Document
import pandas as pd

def substituir_marcadores(documento, dados_marcadores):
    """Percorre o documento e substitui os marcadores nos rodapés, corpo principal e tabelas."""
    for section in documento.sections:
        footer = section.footer
        if footer:
            for paragrafo in footer.paragraphs:
                substituir_texto(paragrafo, dados_marcadores)

    for paragrafo in documento.paragraphs:
        substituir_texto(paragrafo, dados_marcadores)

    for tabela in documento.tables:
        for linha in tabela.rows:
            for celula in linha.cells:
                for paragrafo in celula.paragraphs:
                    substituir_texto(paragrafo, dados_marcadores)

def substituir_texto(paragrafo, dados_marcadores):
    """Substitui os marcadores dentro de um parágrafo, garantindo que runs fragmentados sejam tratados."""
    texto_completo = "".join(run.text for run in paragrafo.runs)

    alterado = False
    for marcador, valor in dados_marcadores.items():
        placeholder = '{' + marcador + '}'
        if placeholder in texto_completo:
            texto_completo = texto_completo.replace(placeholder, str(valor))
            alterado = True
    
    if alterado:
        for run in paragrafo.runs:
            run.text = ""  # Limpa os runs existentes para reescrever o texto corrigido
        paragrafo.runs[0].text = texto_completo  # Atribui o novo texto ao primeiro run

def mesclar_planilhas():
    """Mescla as planilhas "BaseProducao_436.xlsx" e "BaseProducao_553.xlsx", adicionando uma coluna de identificação do contrato."""
    caminho_436 = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\BaseProducao_436.xlsx"
    caminho_553 = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\BaseProducao_553.xlsx"
    caminho_saida = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\BaseProducao_Consolidada.xlsx"

    try:
        df_436 = pd.read_excel(caminho_436)
        df_553 = pd.read_excel(caminho_553)

        df_436["Contrato"] = "016.2022 e 148.2021"
        df_553["Contrato"] = "381.2023"

        df_consolidado = pd.concat([df_436, df_553], ignore_index=True)
        df_consolidado.to_excel(caminho_saida, index=False)

        print(f"Planilha consolidada salva com sucesso em: {caminho_saida}")

    except FileNotFoundError:
        print("Erro: Um ou ambos os arquivos de entrada não foram encontrados.")
    except Exception as e:
        print(f"Ocorreu um erro ao mesclar as planilhas: {e}")

def contar_documentos_por_contrato(data_filtro, contrato):
    """Filtra a planilha consolidada por um intervalo de tempo especificado e contrato, e conta os documentos e guias de emissão."""
    caminho_consolidado = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\BaseProducao_Consolidada.xlsx"

    try:
        df = pd.read_excel(caminho_consolidado)
        df["Data Emissão"] = pd.to_datetime(df["Data Emissão"], errors="coerce")

        mes, ano = map(int, data_filtro.split("/"))
        df_filtrado = df[(df["Data Emissão"].dt.month == mes) & 
                         (df["Data Emissão"].dt.year == ano) & 
                         (df["Contrato"] == contrato)]

        qtd_documentos = df_filtrado["Documento"].count()
        qtd_guias = df_filtrado["Guia de Emissão"].nunique()

        return qtd_documentos, qtd_guias

    except FileNotFoundError:
        print(f"Erro: Arquivo consolidado não encontrado no caminho: {caminho_consolidado}")
        return None, None
    except Exception as e:
        print(f"Ocorreu um erro ao processar os dados: {e}")
        return None, None

def gerar_documento(contrato, qtd_documentos, qtd_guias, dados_entrada):
    """Gera um arquivo Word específico para o contrato informado com as informações calculadas."""
    caminho_documento = r"C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\RelatorioAvanco.docx"
    caminho_salvar = caminho_documento.replace(".docx", f"_{contrato.replace('/', '-')}.docx")

    dados_marcadores = {
        'n_projeto_capa': dados_entrada[contrato]['n_projeto'],
        'n_projeto_tabela': dados_entrada[contrato]['n_projeto'],  # Usando o mesmo valor de n_projeto_capa
        'nome_doc_corsan_rodape': dados_entrada[contrato]['nome_doc_corsan_rodape'],
        'niv_rev_corsan_capa': dados_entrada['niv_rev_corsan'],  # Aplicando o mesmo valor para capa e tabela
        'niv_rev_corsan_tabela': dados_entrada['niv_rev_corsan'],
        'n_doc_engevix_tabela': dados_entrada[contrato]['n_doc_engevix_tabela'],
        'mes_ano_capa': dados_entrada['mes_ano_capa'],
        'niv_rev_engevix_tabela': dados_entrada['niv_rev_engevix_tabela'],
        'n_contrato_tabela': contrato,
        'mes_ano_tabela': dados_entrada['mes_ano_tabela'],
        'n_contrato_txt': contrato,
        'mes_ano_txt': dados_entrada['mes_ano_txt'],
        'qtd_doc': qtd_documentos,
        'qtd_gemi': qtd_guias,
    }

    try:
        documento = Document(caminho_documento)
        substituir_marcadores(documento, dados_marcadores)
        documento.save(caminho_salvar)
        print(f"Relatório para contrato {contrato} salvo com sucesso em: {caminho_salvar}")
    except FileNotFoundError:
        print(f"Erro: Arquivo não encontrado no caminho: {caminho_documento}")
    except Exception as e:
        print(f"Ocorreu um erro ao gerar o documento para o contrato {contrato}: {e}")

def main():
    mes_ano = input("\nDigite o período desejado (MM/YYYY): ")

    dados_entrada = {
        'mes_ano_capa': mes_ano,
        'niv_rev_corsan': input("Nível de Revisão CORSAN (Capa e Tabela): "),  # Solicitado apenas uma vez
        'niv_rev_engevix_tabela': input("Nível de Revisão ENGEVIX (Tabela): "),
        'mes_ano_tabela': mes_ano,
        'mes_ano_txt': mes_ano,
    }

    contratos = ["381.2023", "016.2022 e 148.2021"]
    
    for contrato in contratos:
        print(f"\nInsira os dados para o contrato {contrato}:")
        dados_entrada[contrato] = {
            'n_projeto': input("Número do Projeto (Capa e Tabela): "),
            'nome_doc_corsan_rodape': input("Nome do Documento CORSAN (Rodapé): "),
            'n_doc_engevix_tabela': input("Número do Documento ENGEVIX (Tabela): "),
        }

    mesclar_planilhas()

    for contrato in contratos:
        qtd_documentos, qtd_guias = contar_documentos_por_contrato(mes_ano, contrato)
        if qtd_documentos is not None and qtd_guias is not None:
            gerar_documento(contrato, qtd_documentos, qtd_guias, dados_entrada)

if __name__ == "__main__":
    main()


Digite o período desejado (MM/YYYY):  01/2024
Nível de Revisão CORSAN (Capa e Tabela):  A00
Nível de Revisão ENGEVIX (Tabela):  0A



Insira os dados para o contrato 381.2023:


Número do Projeto (Capa e Tabela):  DOI
Nome do Documento CORSAN (Rodapé):  DOI
Número do Documento ENGEVIX (Tabela):  DOI



Insira os dados para o contrato 016.2022 e 148.2021:


Número do Projeto (Capa e Tabela):  DOU
Nome do Documento CORSAN (Rodapé):  DOU
Número do Documento ENGEVIX (Tabela):  DOU


Planilha consolidada salva com sucesso em: C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\BaseProducao_Consolidada.xlsx
Relatório para contrato 381.2023 salvo com sucesso em: C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\RelatorioAvanco_381.2023.docx
Relatório para contrato 016.2022 e 148.2021 salvo com sucesso em: C:\Users\Dell\OneDrive\Documentos\GitHub\Polatm_RamonVieira\inputs\RelatorioAvanco\RelatorioAvanco_016.2022 e 148.2021.docx
